<a href="https://colab.research.google.com/github/isosafrasaurus/3d-1d/blob/main/3d_1d_drvar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

import sys, os, re
import numpy as np

WORK_PATH = "/content/drive/MyDrive/Research/3d-1d"
SOURCE_PATH = os.path.join(WORK_PATH, "src")
EXPORT_PATH = os.path.join(WORK_PATH, "export")
sys.path.append(SOURCE_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# @title Install all packages
%%capture
!pip install ipywidgets vtk meshio pyvista Rtree

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone --single-branch -b "collapse-iter-dev" "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

!pip install git+https://github.com/dolfin-adjoint/pyadjoint.git --upgrade

| Parameter    | Units                 | Physical Role                                        |
|--------------|-----------------------|------------------------------------------------------|
| $$ \mu $$    | $$\text{Pa}\cdot\text{s}$$ | Dynamic viscosity of blood                           |
| $$ k_t $$    | $$ \text{m}^2 $$      | 3D tissue permeability                               |
| $$ k_v $$    | $$ \text{m}^2 $$      | Axial permeability along the vessel                  |
| $$ \gamma $$ | $$ \text{m}/(\text{Pa}\cdot\text{s}) $$ | Vessel wall permeability coefficient |
| $$ \gamma_R $$ | $$ \text{m}/(\text{Pa}\cdot\text{s}) $$ | Boundary outflow permeability                 |
| $$ \gamma_a $$ | $$\text{m}$$                    | Terminal vessel hydraulic conductance at end |

Velocity units: $$\frac{\text{m}^2}{\text{Pa} \cdot s} \times \frac{\text{Pa}}{\text{m}} = \frac{\text{m}}{\text{s}}$$

In [12]:
from graphnics import FenicsGraph

In [15]:
import tissue, fem, visualize

TEST_GRAPH = FenicsGraph()

TEST_GRAPH_NODES = {
    0: [0.000, 0.020, 0.015],
    1: [0.010, 0.020, 0.015],
    2: [0.022, 0.013, 0.015],
    3: [0.022, 0.028, 0.015],
    4: [0.015, 0.005, 0.015],
    5: [0.015, 0.035, 0.015],
    6: [0.038, 0.005, 0.015],
    7: [0.038, 0.035, 0.015]
}

for node_id, pos in TEST_GRAPH_NODES.items():
    TEST_GRAPH.add_node(node_id, pos=pos)

TEST_GRAPH_EDGES = [
    (0, 1, 0.004),
    (1, 2, 0.003),
    (1, 3, 0.003),
    (2, 4, 0.002),
    (2, 6, 0.003),
    (3, 5, 0.002),
    (3, 7, 0.003)
]

for (u, v, radius) in TEST_GRAPH_EDGES:
    TEST_GRAPH.add_edge(u, v, radius=radius)

Omega_bounds = np.array([[0, 0, 0], [0.08, 0.08, 0.08]])

TEST_MESH = tissue.MeshBuild(
    TEST_GRAPH,
    Omega_bounds = Omega_bounds,
    Omega_mesh_voxel_dim = (64, 32, 32),
    Lambda_num_nodes_exp = 6
)

TEST_SINK_FACE = TEST_MESH.get_Omega_axis_plane("left")

TEST_DOMAIN = tissue.DomainBuild(
    mesh_build = TEST_MESH,
    Lambda_inlet = [0],
    Omega_sink = TEST_SINK_FACE
)

CUBES_TEST = fem.SubCubes(
    domain = TEST_DOMAIN,
    gamma = 1.0e-6,      # (can vary)
    gamma_R = 1.0e-7,    # (can vary)
    gamma_a = 1.0e-7,    # (can vary)
    mu = 1.0e-3,         # Viscosity
    k_t = 1.0e-10,       # Tissue permeability in 3D
    k_v = 1.0e-11,       # Vessel permeability in 1D (can vary)
    P_in = 100 * 133.322,
    p_cvp = 1.0 * 133.322,
    lower_cube_bounds = [[0.0, 0.0, 0.0], [0.01, 0.01, 0.01]],
    upper_cube_bounds = [[0.033, 0.03, 0.01], [0.043, 0.04, 0.02]]
)

CUBES_TEST.print_diagnostics()

Averaging over 448 cells: 100%|██████████| 448/448 [00:00<00:00, 1373.34it/s]


Flow Diagnostics
--------------------------------------------------
Sink Boundary:
  Inflow               : 0
  Outflow              : 3.2089302e-06
  Net Flow (sum)       : 3.2089302e-06
  Net Flow (Dolfin)    : 3.2089302e-06
  --> The 'Net Flow (sum)' should equal 'Net Flow (Dolfin)'.
--------------------------------------------------
Neumann Boundary:
  Net Flow (Dolfin)    : -5.4459613e-21
--------------------------------------------------
Entire Domain Boundary:
  Inflow               : -2.5226167e-07
  Outflow              : 3.4611918e-06
  Net Flow (sum)       : 3.2089302e-06
  Net Flow (Dolfin)    : 3.2089302e-06
  --> The 'Net Flow (sum)' should equal 'Net Flow (Dolfin)'.
--------------------------------------------------
Sum of dsOmegaNeumann and dsOmegaSink (Dolfin):
  Neumann + Sink       : 3.2089302e-06
  --> This should match the net flow over the entire domain boundary.
--------------------------------------------------


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import pytz

default_params = {
    'domain': TEST_DOMAIN,
    'gamma': 1.0e-6,
    'gamma_R': 1.0e-7,
    'gamma_a': 1.0e-7,
    'mu': 1.0e-3,
    'k_t': 1.0e-10,
    'k_v': 1.0e-11,
    'P_in': 100 * 133.322,
    'p_cvp': 1.0 * 133.322,
    'lower_cube_bounds': [[0.0, 0.0, 0.0], [0.01, 0.01, 0.01]],
    'upper_cube_bounds': [[0.033, 0.03, 0.01], [0.043, 0.04, 0.02]]
}

def run_sweep(param_name, values, defaults):
    results = []
    for val in values:
        params = defaults.copy()
        params[param_name] = val
        surface_area = params['domain'].get_surface_area()

        cube_test = fem.SubCubes(**params)
        upper_total = cube_test.compute_upper_cube_flux() / surface_area
        upper_in    = cube_test.compute_upper_cube_flux_in() / surface_area
        upper_out   = cube_test.compute_upper_cube_flux_out() / surface_area
        lower_out   = cube_test.compute_lower_cube_flux_out() / surface_area
        results.append((upper_total, upper_in, upper_out, lower_out))
    return results

def perform_sweeps(num_points=50, subset_params=['k_t', 'gamma', 'gamma_R']):
    sweep_configs = {
        'k_t': {
            'values': np.logspace(-18, -8, num_points),
            'xlabel': "Tissue Permeability k_t",
            'title': "Cube Unit Flux vs. Tissue Permeability k_t",
            'plot_type': 'semilogx',
            'grid_kwargs': {'which': "both", 'ls': "--"}
        },
        'gamma': {
            'values': np.logspace(-15, 3, num_points),
            'xlabel': "gamma",
            'title': "Cube Unit Flux vs. gamma",
            'plot_type': 'semilogx',
            'grid_kwargs': {'which': "both", 'ls': "--"}
        },
        'gamma_R': {
            'values': np.logspace(-15, 3, num_points),
            'xlabel': "gamma_R",
            'title': "Cube Unit Flux vs. gamma_R",
            'plot_type': 'semilogx',
            'grid_kwargs': {'which': "both", 'ls': "--"}
        }
    }

    data = []

    for param in subset_params:
        config = sweep_configs[param]
        values = config['values']
        fluxes = run_sweep(param, values, default_params)

        for val, (upper_total, upper_in, upper_out, lower_out) in zip(values, fluxes):
            data.append({
                'sweep_param': param,
                'sweep_value': val,
                'upper_total': upper_total,
                'upper_in': upper_in,
                'upper_out': upper_out,
                'lower_out': lower_out,
                'xlabel': config['xlabel'],
                'title': config['title'],
                'plot_type': config['plot_type'],
                'grid_kwargs': config['grid_kwargs']
            })

    df = pd.DataFrame(data)
    return df, sweep_configs

cst = pytz.timezone("America/Chicago")
now = datetime.datetime.now(cst)
timestamp = now.strftime("%Y%m%d_%H%M")
output_dir = os.path.join(EXPORT_PATH, f"output_sweeps_{timestamp}")
os.makedirs(output_dir, exist_ok=True)

def plot_results(df, sweep_configs, directory, subset_params=['k_t', 'gamma', 'gamma_R']):
    for param in subset_params:
        config = sweep_configs[param]
        df_param = df[df['sweep_param'] == param]

        fig1, ax1 = plt.subplots(figsize=(10, 6))
        if config['plot_type'] == 'semilogx':
            ax1.semilogx(df_param['sweep_value'], df_param['upper_total'],
                         marker='o', linestyle='-', label="Upper Total Unit Flux")
            ax1.semilogx(df_param['sweep_value'], df_param['upper_in'],
                         marker='s', linestyle='--', label="Upper Unit Flux In")
            ax1.semilogx(df_param['sweep_value'], df_param['upper_out'],
                         marker='^', linestyle='-.', label="Upper Unit Flux Out")
        else:
            ax1.plot(df_param['sweep_value'], df_param['upper_total'],
                     marker='o', linestyle='-', label="Upper Total Unit Flux")
            ax1.plot(df_param['sweep_value'], df_param['upper_in'],
                     marker='s', linestyle='--', label="Upper Unit Flux In")
            ax1.plot(df_param['sweep_value'], df_param['upper_out'],
                     marker='^', linestyle='-.', label="Upper Unit Flux Out")
        ax1.set_xlabel(config['xlabel'])
        ax1.set_ylabel("Upper Cube Unit Flux")
        ax1.set_title(config['title'] + " (Upper Unit Fluxes)")
        ax1.grid(True, **config['grid_kwargs'])
        ax1.legend()

        fig1_filename = os.path.join(directory, f"{param}_upper_sweep.png")
        fig1.savefig(fig1_filename)
        plt.show()
        plt.close(fig1)

        fig2, ax2 = plt.subplots(figsize=(10, 6))
        if config['plot_type'] == 'semilogx':
            ax2.semilogx(df_param['sweep_value'], df_param['lower_out'],
                         marker='o', linestyle='-', label="Lower Unit Flux Out")
        else:
            ax2.plot(df_param['sweep_value'], df_param['lower_out'],
                     marker='o', linestyle='-', label="Lower Unit Flux Out")
        ax2.set_xlabel(config['xlabel'])
        ax2.set_ylabel("Lower Cube Unit Flux")
        ax2.set_title(config['title'] + " (Lower Unit Flux Out)")
        ax2.grid(True, **config['grid_kwargs'])
        ax2.legend()

        fig2_filename = os.path.join(directory, f"{param}_lower_sweep.png")
        fig2.savefig(fig2_filename)
        plt.show()
        plt.close(fig2)

def save_data_to_csv(df, directory, filename="sweep_results.csv"):
    file_path = os.path.join(directory, filename)
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

In [ ]:
cst = pytz.timezone("America/Chicago")
now = datetime.datetime.now(cst)
timestamp = now.strftime("%Y%m%d_%H%M")
output_dir = os.path.join(EXPORT_PATH, f"output_sweeps_{timestamp}")

df_results, sweep_configs = perform_sweeps(num_points=50)
plot_results(df_results, sweep_configs, output_dir)
save_data_to_csv(df_results, output_dir)

[np.float64(0.08), np.float64(0.08), np.float64(0.08)]


Averaging over 448 cells: 100%|██████████| 448/448 [00:00<00:00, 1341.91it/s]


[np.float64(0.08), np.float64(0.08), np.float64(0.08)]


Averaging over 448 cells: 100%|██████████| 448/448 [00:00<00:00, 1351.25it/s]


In [17]:
# df_results, sweep_configs = perform_sweeps(num_points=50)
sweep_configs = {
    'k_t': {
        'values': np.logspace(-18, -8, 50),
        'xlabel': "Tissue Permeability k_t",
        'title': "Cube Unit Flux vs. Tissue Permeability k_t",
        'plot_type': 'semilogx',
        'grid_kwargs': {'which': "both", 'ls': "--"}
    },
    'gamma': {
        'values': np.logspace(-15, 3, 50),
        'xlabel': "gamma",
        'title': "Cube Unit Flux vs. gamma",
        'plot_type': 'semilogx',
        'grid_kwargs': {'which': "both", 'ls': "--"}
    },
    'gamma_R': {
        'values': np.logspace(-15, 3, 50),
        'xlabel': "gamma_R",
        'title': "Cube Unit Flux vs. gamma_R",
        'plot_type': 'semilogx',
        'grid_kwargs': {'which': "both", 'ls': "--"}
    }
}

# Set the output directory (should be the same used when saving the data)
cst = pytz.timezone("America/Chicago")
now = datetime.datetime.now(cst)
timestamp = now.strftime("%Y%m%d_%H%M")
# Or if you know the output_dir path, set it directly, for example:
# output_dir = "/path/to/your/saved/output_dir"
output_dir = os.path.join(EXPORT_PATH, f"output_sweeps_{timestamp}")

df_results = pd.read_csv("data")
plot_results(df_results, sweep_configs, output_dir)
# save_data_to_csv(df_results, output_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'data'